In [1]:
import os
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from sec_edgar_downloader import Downloader

In [2]:
# Constants
#HEADERS = {"User-Agent": "Sude sude.tiras@sabanciuniv.edu"}  # Replace with your details
DOWNLOAD_DIR = "10Q_filings"
YEARS = list(range(2020, 2026))  # 2015 to 2025
CSV_FILE = "companies.csv"  # Update path if needed

In [3]:
# Load the company list from CSV
def load_companies(csv_file):
    """Reads the CSV file and extracts CIK, Ticker, and Company Name"""
    df = pd.read_csv(csv_file, dtype={'CIK': str})  # Ensure CIK remains a string
    df['CIK'] = df['CIK'].astype(str).str.zfill(10)  # Ensure 10-digit format

    df = df.head(500)  # Limit to the first 500 rows
    return df[['CIK', 'Ticker', 'Company Name']].dropna()

In [4]:
print(load_companies(CSV_FILE))

            CIK Ticker                           Company Name
0    0000320193   AAPL                             Apple Inc.
1    0000789019   MSFT                         MICROSOFT CORP
2    0001045810   NVDA                            NVIDIA CORP
3    0001018724   AMZN                         AMAZON COM INC
4    0001652044  GOOGL                          Alphabet Inc.
..          ...    ...                                    ...
495  0000093751    STT                      STATE STREET CORP
496  0001671750  DSCSY                  Disco Corporation/ADR
497  0001015922   CHKP  CHECK POINT SOFTWARE TECHNOLOGIES LTD
498  0001609711   GDDY                           GoDaddy Inc.
499  0001125259    CUK                           CARNIVAL PLC

[500 rows x 3 columns]


In [5]:
# Function to sanitize company names for file saving
def sanitize_filename(name):
    """Removes special characters and spaces from company names for file naming"""
    return "".join(c for c in name if c.isalnum() or c in (" ", "_")).replace(" ", "_")

In [ ]:
# Function to download all 10-Q filings for a given year
def download_10q_filings(cik, ticker, company_name, year):
    """Downloads all available 10-Q filings for a given company and year"""
    try:
        email_address = "sude.tiras@sabanciuniv.edu"  # Replace with your email address
        dl = Downloader(DOWNLOAD_DIR, email_address)
        
        # Create a company-specific directory
        sanitized_name = sanitize_filename(company_name)
        company_dir = os.path.join(DOWNLOAD_DIR, sanitized_name)
        os.makedirs(company_dir, exist_ok=True)
        
        # Download all available 10-Q filings
        filings = dl.get("10-Q", cik, after=f"{year}-01-01", before=f"{year}-12-31")


        if isinstance(filings, int) or not filings:
            # If no 10-Qs are found, try fetching 10-Ks
            filings = dl.get("10-K", cik, after=f"{year}-01-01", before=f"{year}-12-31")

        # Move files to the company-specific directory
        for i, filing_path in enumerate(filings, start=1):
            new_filename = os.path.join(company_dir, f"{sanitized_name}_10Q_{year}_Q{i}.txt")
            os.rename(filing_path, new_filename)

        return f"Downloaded {len(filings)} 10-Q filings for {company_name} ({ticker}) in {year}"

    except Exception as e:
        return f"Failed {company_name} ({ticker}) {year}: {str(e)}"

In [ ]:
# Main function to download all 10-Qs
def main():
    companies = load_companies(CSV_FILE)
    
    with ThreadPoolExecutor(max_workers=1) as executor:  # Multi-threaded downloading
        future_to_company = {}
        for _, row in companies.iterrows():
            cik, ticker, company_name = row['CIK'], row['Ticker'], row['Company Name']
            for year in YEARS:
                future = executor.submit(download_10q_filings, cik, ticker, company_name, year)
                future_to_company[future] = (company_name, ticker, year)

        # Progress bar
        for future in tqdm(as_completed(future_to_company), total=len(future_to_company)):
            company_name, ticker, year = future_to_company[future]
            try:
                result = future.result()
                print(result)
            except Exception as e:
                print(f"Error downloading {company_name} ({ticker}) {year}: {e}")

if __name__ == "__main__":
    main()


  0%|          | 1/3000 [00:03<3:02:04,  3.64s/it]

No 10-Q or 10-K filings found for Apple Inc. (AAPL) in 2020


  0%|          | 2/3000 [00:07<2:54:20,  3.49s/it]

No 10-Q or 10-K filings found for Apple Inc. (AAPL) in 2021


  0%|          | 2/3000 [00:10<4:14:27,  5.09s/it]


In [ ]:
import os
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from sec_edgar_downloader import Downloader

# Constants
HEADERS = {"User-Agent": "sude.tiras@sabanciuniv.edu"}  # Update with your details
DOWNLOAD_DIR = "10Q_filings"
YEARS = list(range(2015, 2026))  # Download filings for 2015-2025
CSV_FILE = "companies.csv"  # Update with correct path

# Load company CIKs from CSV
def load_companies(csv_file):
    df = pd.read_csv(csv_file, dtype={'CIK': str})  # Ensure CIKs are strings
    df['CIK'] = df['CIK'].str.zfill(10)  # Ensure CIKs are 10-digit zero-padded
    return df[['CIK', 'Ticker', 'Company Name']].dropna()

# Sanitize company names for file storage
def sanitize_name(name):
    return "".join(c for c in name if c.isalnum() or c in (" ", "_")).replace(" ", "_")

def download_filings(cik, ticker, company_name, year):
    try:
        dl = Downloader(DOWNLOAD_DIR, "sude.tiras@sabanciuniv.edu")
        print(f"🔍 Checking 10-Q filings for {company_name} ({ticker}) in {year}...")

        # Try fetching 10-Qs first
        filings = dl.get("10-Q", cik, after=f"{year}-01-01", before=f"{year}-12-31")

        if isinstance(filings, int):
            print(f"✅ Successfully downloaded {filings} filings for {company_name} in {year}")
        else:
            print(f"❌ No filings found for {company_name} in {year}")
    
    except Exception as e:
        print(f"❌ Error downloading {company_name} ({ticker}) {year}: {e}")



"""
# Download all 10-Q filings for a company in a given year
def download_filings(cik, ticker, company_name, year):
    try:
        dl = Downloader(DOWNLOAD_DIR, "hitiras@hotmail.com")
        folder = os.path.join(DOWNLOAD_DIR, sanitize_name(company_name))
        os.makedirs(folder, exist_ok=True)

        # Try fetching 10-Qs first
        filings = dl.get("10-Q", cik, after=f"{year}-01-01", before=f"{year}-12-31")

        # If no 10-Qs, attempt to fetch 10-K as a backup
        if not filings or isinstance(filings, int):
            filings = dl.get("10-K", cik, after=f"{year}-01-01", before=f"{year}-12-31")
            if not filings or isinstance(filings, int):
                return f"No filings found for {company_name} ({ticker}) in {year}"

        # Rename and store files
        for i, filing in enumerate(filings, start=1):
            new_filename = os.path.join(folder, f"{sanitize_name(company_name)}_{year}_Q{i}.txt")
            os.rename(filing, new_filename)

        return f"Downloaded {len(filings)} filings for {company_name} ({ticker}) in {year}"
    
    except Exception as e:
        return f"Error downloading {company_name} ({ticker}) {year}: {e}"
"""
# Main function to manage downloads
def main():
    companies = load_companies(CSV_FILE)
    
    with ThreadPoolExecutor(max_workers=5) as executor:  # Control request rate
        futures = {
            executor.submit(download_filings, row['CIK'], row['Ticker'], row['Company Name'], year): 
            (row['Company Name'], row['Ticker'], year)
            for _, row in companies.iterrows()
            for year in YEARS
        }

        for future in tqdm(as_completed(futures), total=len(futures)):
            company_name, ticker, year = futures[future]
            try:
                print(future.result())
            except Exception as e:
                print(f"Error downloading {company_name} ({ticker}) {year}: {e}")

if __name__ == "__main__":
    main()


🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2015...
🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2019...
🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2016...
🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2018...
🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2017...
✅ Successfully downloaded 3 filings for Apple Inc. in 2015
✅ Successfully downloaded 3 filings for Apple Inc. in 2016
✅ Successfully downloaded 3 filings for Apple Inc. in 2018
✅ Successfully downloaded 3 filings for Apple Inc. in 2019
✅ Successfully downloaded 3 filings for Apple Inc. in 2017
🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2021...
🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2020...


  0%|          | 0/106909 [00:00<?, ?it/s]

None
None
None
None
None
🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2022...
🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2023...
🔍 Checking 10-Q filings for Apple Inc. (AAPL) in 2024...


  0%|          | 6/106909 [00:00<4:40:35,  6.35it/s]

✅ Successfully downloaded 3 filings for Apple Inc. in 2020
None
✅ Successfully downloaded 3 filings for Apple Inc. in 2021
None


...

  0%|          | 95/106909 [00:51<10:28:36,  2.83it/s]

✅ Successfully downloaded 0 filings for TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD in 2021
None
✅ Successfully downloaded 0 filings for TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD in 2023
None


  0%|          | 96/106909 [00:51<15:57:49,  1.86it/s]

🔍 Checking 10-Q filings for TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD (TSM) in 2025...
✅ Successfully downloaded 0 filings for TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD in 2022


🔍 Checking 10-Q filings for Tesla, Inc. (TSLA) in 2015...
🔍 Checking 10-Q filings for Tesla, Inc. (TSLA) in 2016...
🔍 Checking 10-Q filings for Tesla, Inc. (TSLA) in 2017...
✅ Successfully downloaded 0 filings for TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD in 2024
✅ Successfully downloaded 0 filings for TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD in 2025
🔍 Checking 10-Q filings for Tesla, Inc. (TSLA) in 2018...
🔍 Checking 10-Q filings for Tesla, Inc. (TSLA) in 2019...
✅ Successfully downloaded 3 filings for Tesla, Inc. in 2015
✅ Successfully downloaded 3 filings for Tesla, Inc. in 2016
🔍 Checking 10-Q filings for Tesla, Inc. (TSLA) in 2020...
✅ Successfully downloaded 3 filings for Tesla, Inc. in 2018
🔍 Checking 10-Q filings for Tesla, Inc. (TSLA) in 2021...
✅ Successfully downloaded 3 filings for Tesla, Inc. in 2017
🔍 Checking 10-Q filings for Tesla, Inc. (TSLA) in 2022...
✅ Successfully downloaded 3 filings for Tesla, Inc. in 2019
✅ Successfully downloaded 3 filings for Tesla, Inc. in 2

In [2]:
from sec_edgar_downloader import Downloader
import os

dl = Downloader("test_downloads", "your.email@example.com")  
cik = "0000320193"  # Apple CIK

# Download 10-Q filings for Apple in 2023
num_files = dl.get("10-Q", cik, after="2023-01-01", before="2023-12-31")

# Check if downloads happened
if isinstance(num_files, int) and num_files > 0:
    print(f"✅ Successfully downloaded {num_files} filings.")
else:
    print("❌ No filings downloaded. Check sec-edgar-downloader.")


✅ Successfully downloaded 3 filings.


In [3]:
import os

# Find the actual save location inside "test_downloads"
download_folder = "test_downloads/sec-edgar-filings/0000320193/10-Q"

if os.path.exists(download_folder):
    print(f"✅ Files are stored in: {download_folder}")
    print("Downloaded files:", os.listdir(download_folder))
else:
    print("❌ No files found in expected location.")


❌ No files found in expected location.


In [4]:
import os

root_folder = "test_downloads"

if os.path.exists(root_folder):
    print(f"📂 Folders inside {root_folder}:")
    print(os.listdir(root_folder))
else:
    print("❌ test_downloads/ directory does not exist.")


❌ test_downloads/ directory does not exist.


In [7]:
from sec_edgar_downloader import Downloader

dl = Downloader("test_downloads", "your.email@example.com")

# Print the actual save directory
print(f"✅ sec-edgar-downloader is saving files in: {download_folder}")


✅ sec-edgar-downloader is saving files in: test_downloads/sec-edgar-filings/0000320193/10-Q


In [5]:
import os

for root, dirs, files in os.walk("test_downloads"):
    for file in files:
        if "10-Q" in file:
            print(f"✅ Found: {os.path.join(root, file)}")


In [1]:
import sec_edgar_downloader
print(sec_edgar_downloader.__version__)


5.0.3


In [3]:
from sec_edgar_downloader import Downloader

# Provide an email address as required by SEC
dl = Downloader("test_downloads", "sude.tiras@sabanciuniv.edu")  

# Apple's CIK
cik = "0000320193"  

# Attempt to download 10-Q for 2023
filings = dl.get("10-Q", cik, after="2023-01-01", before="2023-12-31")

if isinstance(filings, int):
    print(f"✅ Successfully downloaded {filings} 10-Q filings.")
else:
    print("❌ No filings downloaded. Check sec-edgar-downloader.")


✅ Successfully downloaded 3 10-Q filings.


In [2]:
import requests

cik = "0000320193"  # Apple Inc.
headers = {"User-Agent": "your.email@example.com"}
url = f"https://data.sec.gov/submissions/CIK{cik}.json"

response = requests.get(url, headers=headers)
if response.status_code == 200:
    data = response.json()
    filings = data.get("filings", {}).get("recent", {})
    
    # Check if 10-Qs exist
    form_types = filings.get("form", [])
    filing_dates = filings.get("filingDate", [])

    # Filter for 10-Q forms
    ten_qs = [(date, form) for date, form in zip(filing_dates, form_types) if form == "10-Q"]
    
    if ten_qs:
        print("✅ 10-Q Filings Found:")
        for date, form in ten_qs:
            print(f"- {form} filed on {date}")
    else:
        print("❌ No 10-Q filings found in SEC's API response.")
else:
    print("❌ Failed to fetch SEC data.")


✅ 10-Q Filings Found:
- 10-Q filed on 2025-01-31
- 10-Q filed on 2024-08-02
- 10-Q filed on 2024-05-03
- 10-Q filed on 2024-02-02
- 10-Q filed on 2023-08-04
- 10-Q filed on 2023-05-05
- 10-Q filed on 2023-02-03
- 10-Q filed on 2022-07-29
- 10-Q filed on 2022-04-29
- 10-Q filed on 2022-01-28
- 10-Q filed on 2021-07-28
- 10-Q filed on 2021-04-29
- 10-Q filed on 2021-01-28
- 10-Q filed on 2020-07-31
- 10-Q filed on 2020-05-01
- 10-Q filed on 2020-01-29
- 10-Q filed on 2019-07-31
- 10-Q filed on 2019-05-01
- 10-Q filed on 2019-01-30
- 10-Q filed on 2018-08-01
- 10-Q filed on 2018-05-02
- 10-Q filed on 2018-02-02
- 10-Q filed on 2017-08-02
- 10-Q filed on 2017-05-03
- 10-Q filed on 2017-02-01
- 10-Q filed on 2016-07-27
- 10-Q filed on 2016-04-27
- 10-Q filed on 2016-01-27
- 10-Q filed on 2015-07-22
- 10-Q filed on 2015-04-28
- 10-Q filed on 2015-01-28
- 10-Q filed on 2014-07-23


In [3]:
import os

download_dir = "10Q_filings"  # Update if different
company_name = "Apple_Inc"  # Adjust if needed

company_folder = os.path.join(download_dir, company_name)
if os.path.exists(company_folder):
    print(f"✅ Files downloaded in: {company_folder}")
    print("Downloaded files:", os.listdir(company_folder))
else:
    print("❌ No files found. Check sec-edgar-downloader settings.")


✅ Files downloaded in: 10Q_filings\Apple_Inc
Downloaded files: []
